In [47]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import numba

from my_timer import timer_decorator

# funkcionály optimalizované pro numpy

In [48]:
p, a, b = 3, -1, 1
ne = 200
x = np.linspace(a, b, ne + 1)
h = np.diff(x)


def energy_numpy(v_internal, fx, v, h, p):
    v[1:-1] = v_internal
    vx = (v[1:] - v[:-1]) / h
    v_mid = (v[1:] + v[:-1]) / 2
    Jv_density = (1 / p) * np.abs(vx)**p - fx * v_mid

    return np.sum(h * Jv_density)


# exact minimizer for p=2
def u_init(x):
    return 0 * (x + 1) * (x - 1)


# rhs
def f(x):
    return -10 * np.ones(x.size)


v = u_init(x)            # testing function
v_internal = v[1:-1].copy()

x_mid = (x[1:] + x[:-1]) / 2
fx = f(x_mid)

# minimalizace numpy implementace
cca 3x rychlejší než bez numpy

In [49]:

minimize_timed = timer_decorator(minimize)

print("energy (init)=", energy_numpy(v_internal, fx, v, h, p))


solopt = minimize_timed(energy_numpy, v_internal, args=(fx, v, h, p))

print("energy (final)=", energy_numpy(solopt.x, fx, v, h, p))
print(solopt.nit)

energy (init)= 0.0
minimize#0: 7.134 s
energy (final)= -16.865224260083938
214


# numba
4s vs 7s u mně, tedy asi o 75% rychlejší

In [53]:
from numba import jit, float64, boolean, int64, prange
import numpy as np

@jit(nopython=True)
def energy_numba(v_internal, fx, v, h, p):
    v[1:-1] = v_internal
    n = h.shape[0]
    Jv_density = 0.0
    for i in range(n):
        Jv_density += h[i] * ((1 / p) * np.abs((v[i+1] - v[i]) / h[i])**p - fx [i]* (v[i+1] + v[i]) / 2)
    return Jv_density


In [54]:
print("energy (init)=", energy_numba(v_internal, fx, v, h, p))


solopt = minimize_timed(energy_numba, v_internal, args=(fx, v, h, p))

print("energy (final)=", energy_numba(solopt.x, fx, v, h, p))
print(solopt.nit)

energy (init)= 0.0
minimize#2: 4.109 s
energy (final)= -16.86522425995996
211


# jemnější síť

In [58]:
p, a, b = 3, -1, 1
ne = 500
x = np.linspace(a, b, ne + 1)
h = np.diff(x)

v = u_init(x)            # testing function
v_internal = v[1:-1].copy()

x_mid = (x[1:] + x[:-1]) / 2
fx = f(x_mid)

In [59]:

print("energy (init)=", energy_numpy(v_internal, fx, v, h, p))


solopt = minimize_timed(energy_numpy, v_internal, args=(fx, v, h, p))

print("energy (final)=", energy_numpy(solopt.x, fx, v, h, p))
print(solopt.nit)

energy (init)= 0.0
minimize#3: 47.421 s
energy (final)= -16.865438707319285
507


In [60]:
print("energy (init)=", energy_numba(v_internal, fx, v, h, p))


solopt = minimize_timed(energy_numba, v_internal, args=(fx, v, h, p))

print("energy (final)=", energy_numba(solopt.x, fx, v, h, p))
print(solopt.nit)

energy (init)= 0.0
minimize#4: 27.905 s
energy (final)= -16.865438729205
509


numba byla opět 1.75x rychlejši 47s vs 27s